In [ ]:
# Install dependencies
!pip install -q transformers accelerate gradio

# Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr


In [ ]:
# Load the fastest small open-source model (Phi-2)
model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

In [ ]:
# Define AI response function
def ai_pair_programmer(task_type, code_context):
    prompt = f"""You are an expert AI Pair Programmer.
Your task is: {task_type}
Here is the code to work with:
{code_context}

Provide your detailed response:"""

    result = pipe(prompt, max_new_tokens=100, temperature=0.3, do_sample=False)
    output = result[0]["generated_text"].replace(prompt, "").strip()
    return output

# Build clean & impressive Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚀 AI Pair Programmer")
    gr.Markdown("💡 Paste your code & describe your task to receive expert suggestions instantly.")

    with gr.Row():
        task_type = gr.Textbox(label="🔧 Task", placeholder="e.g. refactor, fix bugs, optimize, explain", scale=1)
        run_button = gr.Button("🚀 Generate", scale=0)

    code_context = gr.Code(label="💻 Code Input", language="python", lines=15)
    output = gr.Code(label="🧠 AI Output", language="python", lines=15)

    run_button.click(ai_pair_programmer, inputs=[task_type, code_context], outputs=output)

# Launch Gradio app
demo.launch()